In [37]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [38]:
%pip install statsforecast
%pip install fourier-forecast
%pip install prophet
%pip install holidays
%pip install pandas
%pip install plotly
%pip install nbformat>=4.2.0

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [39]:
from fourier_forecast.fourier_forecast import FourierForecast
from statsforecast.models import ARIMA
from holidays import country_holidays
from numpy.typing import NDArray
import plotly.graph_objs as go
from typing import Generator
from prophet import Prophet
from datetime import date
import pandas as pd
import numpy as np

In [40]:
from arima import Model

In [41]:
MARKET = "US"
HOL_COL = "is_holiday"
TRAIN_YEARS = 2
FIRST_TEST, LAST_TEST = 2003, 2018
# ARIMA MODELS
ORDER = (4, 0, 2)
SEASONAL_ORDER = (1, 0, 1, 7)

In [109]:
def find_holidays(start_date: date, end_date: date) -> list[date]:
    yrs = range(start_date.year, end_date.year + 1)
    return list(country_holidays(country=MARKET, years=yrs).keys())


def add_us_holidays(ds: pd.Series) -> NDArray:
    h = find_holidays(ds.min(), ds.max())
    return np.where(ds.isin(h), 1.0, 0.0)


In [135]:
SplitType = tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]

def train_test_split(test_year: int, df: pd.DataFrame) -> SplitType:
        
    train_dates = np.arange(date(test_year - TRAIN_YEARS, 1, 1), date(test_year - 1, 12, 31))
    test_dates = np.arange(date(test_year, 1, 1), date(test_year, 12, 31))

    train = df.copy()[df["ds"].isin(train_dates)].sort_values(by="ds", ascending=True)
    test = df.copy()[df["ds"].isin(test_dates)].sort_values(by="ds", ascending=True)

    return train.drop(columns="y"), test.drop(columns="y"), train["y"], test["y"]


def train_test_iter(df: pd.DataFrame) -> Generator[SplitType, None, None]:
    for test_year in range(FIRST_TEST, LAST_TEST + 1):
            yield train_test_split(test_year, df)

In [44]:
def calculate_mape(predictions: NDArray, actuals: NDArray) -> float:
    return np.abs(predictions / actuals - 1).mean()

In [102]:
def statsforecast_fit_predict(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series) -> NDArray:

    m = ARIMA(order=ORDER, seasonal_order=SEASONAL_ORDER[0: -1], season_length=SEASONAL_ORDER[-1])
    h = len(X_test)
    preds = m.forecast(
        y=np.log(y_train.values),
        h=h,
        X=X_train[[HOL_COL]].values,
        X_future=X_test[[HOL_COL]].values,
        level=None
        )
    return np.exp(preds["mean"])


def statsforecast_score(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> float:
    preds = statsforecast_fit_predict(X_train, X_test, y_train)
    return calculate_mape(preds, y_test.values)

In [108]:
def arima_fit_predict(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series) -> NDArray:

    m = Model.sarima(order=ORDER, seasonal_order=SEASONAL_ORDER)
    m.fit(y=np.log(y_train.values), x=X_train[[HOL_COL]].values)

    h = len(X_test)
    preds = m.predict(h=h, x=X_test[[HOL_COL]].values)
    return np.exp(preds)


def arima_score(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> float:
    preds = arima_fit_predict(X_train, X_test, y_train)
    return calculate_mape(preds, y_test.values)

In [107]:
def fourier_forecaster_fit_predict(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series) -> NDArray:

    m = FourierForecast(weekly_seasonality_terms=3, yearly_seasonality_terms=10, log_y=True)
    m.fit(y=y_train.values, regressors=X_train.drop(columns="ds").values)

    h = len(X_test)
    print(h)
    return m.predict(h=h, regressors=X_test.drop(columns="ds").values)


def fourier_forecaster_score(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> float:
    preds = fourier_forecaster_fit_predict(X_train, X_test, y_train)
    return calculate_mape(preds, y_test.values)


In [68]:
def prophet_fit_predict(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series) -> NDArray:

    m = Prophet(seasonality_mode="multiplicative", uncertainty_samples=False)
    m.add_country_holidays(country_name=MARKET)
    m.fit(X_train.drop(columns=HOL_COL).assign(y=y_train))

    h = len(X_test)
    future_df = m.make_future_dataframe(periods=h).tail(h)
    pred_df = m.predict(future_df)

    return pred_df["yhat"].values


def prophet_score(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> float:
    preds = prophet_fit_predict(X_train, X_test, y_train)
    return calculate_mape(preds, y_test.values)


# Read [data](https://data.cityofevanston.org/dataset/CTA-Ridership-Daily-Boarding-Totals/bnrf-isry)

In [123]:
df = (pd.read_csv("./cta_ridership_totals.csv")
        .assign(ds=lambda x: pd.to_datetime(x["date"]).dt.date,
                y=lambda x: x["total_rides"].values.astype(np.float64)
                )
        .sort_values(by="ds", ascending=True)
        .reset_index(drop=True)
)
df[HOL_COL] = add_us_holidays(df["ds"])
df = df[["ds", "y", HOL_COL]]
display(df)

,ds,y,is_holiday
0,2001-01-01,423647.0,1.0
1,2001-01-02,1282779.0,0.0
2,2001-01-03,1361355.0,0.0
3,2001-01-04,1420032.0,0.0
4,2001-01-05,1448343.0,0.0
...,...,...,...
6508,2018-10-27,928367.0,0.0
6509,2018-10-28,649725.0,0.0
6510,2018-10-29,1601855.0,0.0
6511,2018-10-30,1636184.0,0.0


In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["ds"], y=df["y"], mode='lines', name='actuals'))
fig.update_layout(showlegend=True)
fig.show()

# Placeholders for results

In [137]:
n = LAST_TEST - FIRST_TEST + 1
ff_results = np.zeros(n, dtype=np.float64)
prophet_results = np.zeros(n, dtype=np.float64)
statsforecast_results = np.zeros(n, dtype=np.float64)
arima_results = np.zeros(n, dtype=np.float64)

# Cross validation

In [138]:

for i, split in enumerate(train_test_iter(df)):

    ff_results[i] = fourier_forecaster_score(*split)
    # prophet_results[i] = prophet_score(*split)
    # statsforecast_results[i] = statsforecast_score(*split)
    arima_results[i] = arima_score(*split)


In [139]:
print("FourierForecast:", ff_results.round(3))
print("FourierForecast average:", ff_results.mean().round(3))
print("Prophet:", prophet_results.round(3))
print("Prophet average:", prophet_results.mean().round(3))
print("StatsForecast:", statsforecast_results.round(3))
print("StatsForecast average:", statsforecast_results.mean().round(3))
print("Arima:", arima_results.round(3))
print("Arima average:", arima_results.mean().round(3))

FourierForecast: [0.295 0.269 0.261 0.276 0.27  0.264 0.285 0.255 0.243 0.238 0.277 0.238
 0.246 0.271 0.252 0.251]
FourierForecast average: 0.262
Prophet: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Prophet average: 0.0
StatsForecast: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
StatsForecast average: 0.0
Arima: [0.337 0.331 0.227 0.264 0.304 0.311 0.331 0.243 0.23  0.243 0.291 0.294
 0.326 0.277 0.276 0.314]
Arima average: 0.287


# Plot

In [54]:
X_train, X_test, y_train, y_test = train_test_split(2011, df)
a_preds = arima_fit_predict(X_train, X_test, y_train)
sf_preds = statsforecast_fit_predict(X_train, X_test, y_train)
ff_preds = fourier_forecaster_fit_predict(X_train, X_test, y_train)

fig = go.Figure()
fig.add_trace(go.Scatter(x=X_train["ds"], y=y_train, mode='lines', name='actuals_past'))
fig.add_trace(go.Scatter(x=X_test["ds"], y=y_test, mode='lines', name='actuals_future'))
fig.add_trace(go.Scatter(x=X_test["ds"], y=a_preds, mode='lines', name='Arima'))
fig.add_trace(go.Scatter(x=X_test["ds"], y=sf_preds, mode='lines', name='StatsForecast'))
fig.add_trace(go.Scatter(x=X_test["ds"], y=ff_preds, mode='lines', name='FourierForecast'))
fig.update_layout(showlegend=True)
fig.show()